# Survey of Consumer Finances, Data Extraction

This notebook will get the links to the SCL 2016 files from a Metatab package and build
a dataset we can use for analysis.


In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from pathlib import Path

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Survey of Consumer Finances, Inheritance Extract
`federalreserve.gov-consumer_finances-2016-inherit-1.1.1` Last Update: 2020-08-19T16:15:55

_An Extract from the survey of consumer finances for studying networth and ineritance_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong><a href="https://www.federalreserve.gov/econres/files/scf2016s.zip#p16i6.dta">scf_public_16</a></strong>. Survey of consumer finances, full public dataset, 2016</li><li> <strong><a href="https://www.federalreserve.gov/econres/files/scfp2016s.zip#rscfp2016.dta">scf_extract_16</a></strong>. Survey of consumer finances, public extract, 2016</li><li> <strong><a href="https://www.federalreserve.gov/econres/files/scf2016rw1s.zip#p16_rw1.dta">scf_weights_16</a></strong>. Survey of consumer finances, replicate weights, 2016</li><ul>

In [3]:
# Main variables
scf = pkg.reference('scf_public_16').dataframe()

# Extract file. The variables in this file are created by a SAS macro, which is
# the best (only?) documentation for the meaning of the variable
# https://www.federalreserve.gov/econres/files/bulletin.macro.txt

scfe = pkg.reference('scf_extract_16').dataframe()
scfe.columns = [c.lower() for c in scfe.columns]

mm_cols = [ f"MM{i}" for i in range(1,1000) ]
wt_cols = [ f"WT1B{i}" for i in range(1,1000)]


# Extract and rename a set of columns we are interested in 
cols = {
    'case_id': 'YY1',
    'record_id': 'Y1',
    'age_1': 'X14', # Reconciled age
    'age_2': 'X19', # Reconciled age
    'hisp': 'X7004', # Do you consider yourself to be Hispanic or Latino in culture or origin?
    'race': 'X6809', # Race of respondent
    'addtional_race': 'X6810', # Respondent offered another race categot (1) or did not (5)
    # X6402 #In 2015, did (other adult) receive any income from wages or salaries?
    #'income': 'X5729', # How much was the total income you (and your family living here) received in 2015 from all sources, before taxes and other deductions were made?
    'unusual_income': 'X7650', # Is this income unusually high or low...
    'ed_1': 'X5931', # What is the highest level of school completed or the highest degree you have received?
    'ed_2': 'X6111', # What is the highest level of school completed or the highest degree you have received?
    'ed_mother_1': 'X6032', # What is the highest level of school or the highest degree mother completed?
    'ed_father_1': 'X6033', # What is the highest level of school or the highest degree father completed?
    'ed_mother_2': 'X6132', # What is the highest level of school or the highest degree mother completed?
    'ed_father_2': 'X6133', # What is the highest level of school or the highest degree father completed?  
    'occ_1': 'X7401', # What is the official title of your job?
    'occ_2': 'X7411', # What is the official title of your job?
    'gi_other_value': 'X5818', #How much altogether were any others (inheritances) you have received?
    'fin_risk': 'X7557', # Willingness to take fnancial risks, 1 to 10
    'shop_credit' : 'X7561', # Financial shopping
    'shop_credit_2' : 'X7562', # Financial Shopping
    'fin_know': 'X7556', # Financial Knowledge
    'borrow_vacation': 'X402', # Borrow for a vacation
    'plan_horizon': 'X3008', # which of the time periods listed on this page is most important to you
    'spend_exceeds':'X7510', # would you say that your (family's) spending exceeded your (family's) income,
    'spend_exceeds_excl_house':'X7508', # Spending exceeds, after purchase of house. 
    'wt0': 'X42001' # Weight
    
}

# There are three ( four actually) set os variables for gifts and transfers, 
# We'll ignore the fourth, because it is rare. 

gi_cols_packed = {
    'gi_type':  'X5803 X5808 X5813'.split(), # Was that an inheritance, a trust, or something else?
    'gi_value': 'X5804 X5809 X5814'.split(), # What was its approximate value at the time it was received?
    'gi_year':  'X5805 X5810 X5815'.split(), # In what year was it received?
    'gi_from':  'X5806 X5811 X5816'.split(), # From whom was it received?
}

gi_cols = {}
for i in range(1,4):
    for k, c in gi_cols_packed.items():
        cols[f"{k}_{i}"] = c[i-1]

# Map the column names

df =  scf[cols.values()].rename(columns={v:k for k,v in cols.items()})

# Sum up all of the gifts/transfers. 
df['gi_sum'] = df[['gi_value_1', 'gi_value_2', 'gi_value_3']].sum(axis=1)
df['gi_sum'] = df.gi_sum.mask(df.gi_sum<0, 0) # Remove -9 and -1 values 


# make the implicate number
df.insert(2, 'implicate_id', df.record_id - df.case_id.astype('int32')*10)   
    


In [4]:
# Create more meaningful categoricals for some variables. 

df['race'] = df['race'].astype('category').cat.rename_categories({-7: 'other', 1: 'white', 2: 'black', 3: 'hisp', 4: 'asian', 5: 'aian', 6: 'nhpi'})
    
for c in "gi_type_1 gi_type_2 gi_type_3".split():
    df[c] = df[c].astype('category').cat.rename_categories({0: 'na', 1: 'inheritance', 2: 'trust', 3: 'gift'})

bc = pd.CategoricalDtype(ordered=True)

for c in "ed_1 ed_2 ed_mother_1 ed_mother_2 ed_father_1 ed_father_2".split():
    df[c] = df[c].astype(bc).cat.rename_categories(
        {-1: 'lt_grade_1', 0: 'na', 1: 'grade_1_4', 2: 'grade_5_6', 3: 'grade_7_8', 4: 'grade_9', 5: 'grade_10', 6: 'grade_11', 7: 'grade_12', 
         8: 'hs', 9: 'some_college', 10: 'assoc_vocational', 11: 'assoc_academic', 12: 'bachelors', 13: 'masters', 14: 'advanced'})

df['education'] = df[['ed_1','ed_2']].replace({0:7}).min(axis=1)

df['occ'] = df[['occ_1','occ_2']].replace({0:7}).min(axis=1)


In [5]:
# Extract some variables from the public extract file. These variables
# already have sensible names. 
scf_ext = scfe[['y1','networth',  'income', 'nwcat', 'nwpctlecat', 
                'norminc', 'ninccat',  'ninc2cat', 'nincpctlecat',
                'occat1', 'occat2', 'edcl', 'lifecl', 'famstruct', 'married',  'agecl', 'housecl', 'racecl','racecl4',
                'asset', 'liq','bond', 'fin','nfin', 'debt', 'indcat', 'equity','homeeq', 'revpay','bnkruplast5', 'debt2inc', 'hsaving' , 'saved'
               ]].rename(columns={'y1':'record_id'})

df = df.merge(scf_ext, on='record_id')

# Long-term planners, for plan_horizon/X3008 answered "next few years" or longer
df['lt_planner'] = df.plan_horizon.isin([3,4,5]).astype(int)

df['agecl'] = df.agecl.astype('category').cat.rename_categories({1:'<35', 2:'35-44', 3:'45-54', 4:'55-64', 5:'65-74', 6:'>=75'})
df['edcl'] = df.edcl.astype('category').cat.rename_categories({1:'No HS', 2:'HS/GED', 3:'Some College', 4:'College'})

# Income and networth percentiles
df['nincpctle'] = df.nincpctlecat.astype(bc).cat.rename_categories([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99])
df['ninc'] = df.ninccat.astype(bc).cat.rename_categories([0,20,40,60,80,90])
df['nwpctle'] = df.nwpctlecat.astype(bc).cat.rename_categories([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99])


In [6]:
# Create summary variables for the number of each type of gift/transfer. This part is expensive. 
t = df[['case_id','record_id',
        'gi_type_1', 'gi_type_2', 'gi_type_3',
        'gi_value_1', 'gi_value_2', 'gi_value_3',
        'gi_year_1', 'gi_year_2', 'gi_year_3'
       ]].set_index(['case_id','record_id'])

def n_inherit_types(r):
    return r[['gi_type_1', 'gi_type_2', 'gi_type_3']].value_counts()

nit = df.apply(n_inherit_types, axis=1).fillna(0)

df['n_gift'] = nit.gift.astype(int)
df['n_inherit'] = nit.inheritance.astype(int)
df['n_trust'] = nit.trust.astype(int)
df['n_transfer'] = df['n_gift'] + df['n_inherit'] + df['n_trust']

df['any_transfer'] = (df.n_transfer > 0).astype(int)
df['any_inherit'] = (df.n_inherit > 0).astype(int)



In [11]:
df.to_csv('../data/inherit_scf_16.csv', index=False) 